In [1]:
import sys, os

In [2]:
# Add the directory containing oceandirect to the Python path
oceandirect_path = 'C:/Program Files/Ocean Optics/OceanDirect SDK/Python/'
if os.path.exists(oceandirect_path):
    sys.path.append(os.path.abspath(oceandirect_path))
    print(f"Path exists and added to sys.path: {oceandirect_path}")
else:
    print(f"Path does not exist: {oceandirect_path}")

Path exists and added to sys.path: C:/Program Files/Ocean Optics/OceanDirect SDK/Python/


In [5]:
# try:
#     from oceandirect.OceanDirectAPI import OceanDirectAPI, OceanDirectError, Spectrometer
#     print("Import successful")
# except ImportError as e:
#     print(f"Import failed: {e}")

Import successful


In [3]:
os.getcwd()

'c:\\Users\\Willi\\Documents\\GitHub\\HIRAX\\Ocean_Insight'

In [3]:
from multiprocessing_read_spectra import read_all_serial_numbers, readSpectra

In [25]:
serial_number = read_all_serial_numbers()
output = []

In [26]:
# readSpectra(serial_number, exposure time in microseconds, number of scans, output countaining the full spectrum). Will return select wavelength values. 
readSpectra(serial_number[0], 10000, 1, output)

Device count: 1
Device IDs: [11]
Device serial number: HR4P0848
Started reading spectra with serial#: HR4P0848
serial#/spectra:  HR4P0848 = 18, 4, 10, 3 

Closing device with serial#: HR4P0848


In [27]:
print(output)

[('HR4P0848', [-11.97973204454383, -11.97973204454383, -11.97973204454383, 13.980598970432181, -14.974128930199177, 0.0, 17.97587690879078, 9.985683230953544, -1.996858013622412, 2.995462070968907, -20.96226937908974, 2.995462070968907, 1.9969514430603268, 1.9969514430603268, -3.993622181912042, 5.991132959514406, -11.97973204454383, 3.993995900783874, -6.988591561816209, -13.976020708421062, -3.993622181912042, 25.96750639411167, -0.9984407114581003, 17.97587690879078, -1.996858013622412, -0.9984407114581003, 0.9984640688000767, 18.974752605539855, 1.9969514430603268, 0.0, 11.983095621926617, 4.992552880833138, -4.9919689437832035, -1.996858013622412, 5.991132959514406, 6.989736085295127, 7.98836220671244, 0.9984640688000767, 2.995462070968907, -13.976020708421062, -2.995251854471057, 11.983095621926617, 9.985683230953544, 16.977023646171126, 20.972571098972306, 9.985683230953544, 4.992552880833138, -2.995251854471057, 3.993995900783874, -1.996858013622412, 10.984378031199588, 18.9747

In [33]:
output[0]

('HR4P0848',
 [-11.97973204454383,
  -11.97973204454383,
  -11.97973204454383,
  13.980598970432181,
  -14.974128930199177,
  0.0,
  17.97587690879078,
  9.985683230953544,
  -1.996858013622412,
  2.995462070968907,
  -20.96226937908974,
  2.995462070968907,
  1.9969514430603268,
  1.9969514430603268,
  -3.993622181912042,
  5.991132959514406,
  -11.97973204454383,
  3.993995900783874,
  -6.988591561816209,
  -13.976020708421062,
  -3.993622181912042,
  25.96750639411167,
  -0.9984407114581003,
  17.97587690879078,
  -1.996858013622412,
  -0.9984407114581003,
  0.9984640688000767,
  18.974752605539855,
  1.9969514430603268,
  0.0,
  11.983095621926617,
  4.992552880833138,
  -4.9919689437832035,
  -1.996858013622412,
  5.991132959514406,
  6.989736085295127,
  7.98836220671244,
  0.9984640688000767,
  2.995462070968907,
  -13.976020708421062,
  -2.995251854471057,
  11.983095621926617,
  9.985683230953544,
  16.977023646171126,
  20.972571098972306,
  9.985683230953544,
  4.99255288083

In [34]:
len(output)

1

In [35]:
type(output)

list

In [36]:
from itertools import chain

# Assuming output is a list of lists
if isinstance(output[0], list):
    output = list(chain.from_iterable(output))

print(len(output))

1


In [19]:
output2 = []

In [23]:
output2.append((1, 2))

In [24]:
print(output2)

[(1, 2), (1, 2)]
